In [24]:
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import requests
import plotly.express as px

pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: f'%.{2}f' % x)
pd.set_option('display.max_colwidth', None)

In [25]:
url = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"
response = requests.get(url)
data = response.content

In [26]:
df1 = pd.read_xml(data)
df1.drop(['id', 'terminalName', 'installed', 'locked',
          'installDate', 'removalDate', 'temporary'], axis=1, inplace=True)
df1.head()

,name,lat,long,nbBikes,nbStandardBikes,nbEBikes,nbEmptyDocks,nbDocks
0,"River Street , Clerkenwell",51.53,-0.11,1,1,0,18,19
1,"Phillimore Gardens, Kensington",51.50,-0.20,37,36,1,0,37
2,"Christopher Street, Liverpool Street",51.52,-0.08,3,2,1,29,32
3,"St. Chad's Street, King's Cross",51.53,-0.12,1,1,0,22,23
4,"Sedding Street, Sloane Square",51.49,-0.16,5,4,1,22,27


In [27]:
df2 = pd.read_csv('21Oct2020-27Oct2020.csv')
df2.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,103082433,4080,1940,21/10/2020 01:08,128,"Emperor's Gate, South Kensington",21/10/2020 00:00,128,"Emperor's Gate, South Kensington"
1,103082432,900,8831,21/10/2020 00:15,831,"St John's Crescent, Brixton",21/10/2020 00:00,272,"Baylis Road, Waterloo"
2,103082438,600,16646,21/10/2020 00:10,688,"Northfields, Wandsworth",21/10/2020 00:00,615,"Finlay Street, Fulham"
3,103082437,360,13237,21/10/2020 00:06,135,"Clerkenwell Green, Clerkenwell",21/10/2020 00:00,24,"British Museum, Bloomsbury"
4,103129443,480,15737,21/10/2020 00:08,308,"Long Lane , Bermondsey",21/10/2020 00:00,548,"Westminster Bridge Road, Elephant & Castle"


In [28]:
df = df2.merge(df1, left_on='EndStation Name', right_on='name')
df.drop(['name'], inplace=True, axis=1)
df.columns.values[9:] = 'EndStation ' + df.columns.values[9:]


df = df.merge(df1, left_on='StartStation Name', right_on='name')
df.drop(['name'], inplace=True, axis=1)
df.columns.values[16:] = 'StartStation ' + df.columns.values[16:]

df.set_index(['Rental Id'], inplace=True)
df.head()

,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndStation lat,EndStation long,EndStation nbBikes,EndStation nbStandardBikes,EndStation nbEBikes,EndStation nbEmptyDocks,EndStation nbDocks,StartStation lat,StartStation long,StartStation nbBikes,StartStation nbStandardBikes,StartStation nbEBikes,StartStation nbEmptyDocks,StartStation nbDocks
Rental Id,,,,,,,,,,,,,,,,,,,,,,
103082433,4080,1940,21/10/2020 01:08,128,"Emperor's Gate, South Kensington",21/10/2020 00:00,128,"Emperor's Gate, South Kensington",51.50,-0.19,12,12,0,10,22,51.50,-0.19,12,12,0,10,22
103084263,1020,16425,21/10/2020 08:17,128,"Emperor's Gate, South Kensington",21/10/2020 08:00,128,"Emperor's Gate, South Kensington",51.50,-0.19,12,12,0,10,22,51.50,-0.19,12,12,0,10,22
103110876,960,17182,22/10/2020 12:06,128,"Emperor's Gate, South Kensington",22/10/2020 11:50,128,"Emperor's Gate, South Kensington",51.50,-0.19,12,12,0,10,22,51.50,-0.19,12,12,0,10,22
103123870,2880,16848,22/10/2020 18:35,128,"Emperor's Gate, South Kensington",22/10/2020 17:47,128,"Emperor's Gate, South Kensington",51.50,-0.19,12,12,0,10,22,51.50,-0.19,12,12,0,10,22
103133698,1800,13360,22/10/2020 23:32,128,"Emperor's Gate, South Kensington",22/10/2020 23:02,128,"Emperor's Gate, South Kensington",51.50,-0.19,12,12,0,10,22,51.50,-0.19,12,12,0,10,22


In [23]:
df.to_csv("21Oct2020-27Oct2020new.csv")